In [1]:
import pandas as pd
from gensim.models.fasttext import FastText
from gensim.models import KeyedVectors
import re
import os
import shutil
import sddk
import pickle

In [2]:
metadata_table_long = pd.read_json("../data/metadata_table_long.json")
metadata_table_long.head(5)

,Author,Full title,In,Year,Place,Publisher/Printer,Era,Form/Genre,Discipline/Content,Original,...,ids,id,date_min,date_max,filename,file_year,sents_n,tokens_n,disciplines_list,science_class
0,"Achrelius, Daniel",Scientiarum magnes recitatus publice anno 1690...,None,1690,[Turku],Wall,17th century,Oration,"Mathematics, Astronomy/Astrology/Cosmography, ...",Scientiarum magnes(Google Books),...,[705665],705665,1690.0,1690.0,"Achrelius,_Daniel_-_Scientiarum_magnes__Turku_...",1690.0,630,8839,"[Mathematics, Astronomy/Astrology/Cosmography,...",mixture
1,"Acidalius, Valens","Ad Iordanum Brunum Nolanum, Italum","Poematum Iani Lernutii, Iani Gulielmi, Valenti...",1603,"Liegnitz, Wrocław","Albert, David",17th century,Panegyric poem,Astronomy/Astrology/Cosmography,Ad Iordanum Brunum (1603)(CAMENA)Ad Iordanum B...,...,[801745],801745,1603.0,1603.0,Janus_Lernutius_et_al__-_Poemata__Liegnitz_160...,1603.0,6527,82445,[Astronomy/Astrology/Cosmography],exact_sciences
2,"Acosta, José de",De natura novi orbis libri duo et De promulgat...,None,1589,Salamanca,Guillelum Foquel,16th century,Monograph,"Astronomy/Astrology/Cosmography, Geography/Car...",De natura novi orbis(Biodiversity Heritage Lib...,...,[713323],713323,1589.0,1589.0,"Acosta,_José_de_-_De_natura_novi_orbis__Salama...",1589.0,8905,170205,"[Astronomy/Astrology/Cosmography, Geography/Ca...",exact_sciences
3,"Adam, Melchior","Vitae Germanorum medicorum, qui saeculo superi...",None,1620,Heidelberg,"Rosa, Geyder",17th century,Biography,Medicine,Vitae Germanorum medicorum(MDZ)Alternative lin...,...,[693148],693148,1620.0,1620.0,"Adam,_Melchior_-_Vitae_Germanorum_medicorum__H...",1620.0,17961,193272,[Medicine],life_sciences
4,"Addison, Joseph",Ad insignissimum virum dominum Thomam Burnettu...,"Examen poeticum duplex, sive, Musarum anglican...",1698,London,Richard Wellington I.,17th century,Panegyric poem,Meteorology/Earth sciences,Ad Burnettum sacrae theoriae telluris auctorem...,...,[769230],769230,1698.0,1698.0,Examen_poeticum_duplex__London_1698_pdf.txt,1698.0,3533,47878,[Meteorology/Earth sciences],exact_sciences


In [3]:
filtered_vocab_df = pd.read_json("../data/filtered_vocab_df.json")
filtered_vocab_df.head(20)

,word,1501-1550,1551-1600,1601-1650,1651-1700,mean,in_lila_embeddings,in_lasla,in_operamaiora,transl
3299,dico,43255,122549,42457,72224,70121.25,True,True,True,"say, call, tell"
960,pars,33225,93059,44337,52385,55751.50,True,True,True,part
3871,possum,25029,82700,38391,76841,55740.25,True,True,True,"be able, can"
6039,habeo,31690,89199,39443,59770,55025.50,True,True,True,"have, hold, possess, consider, think"
2898,facio,31814,98524,36944,51438,54680.00,True,True,True,"do, make, handle"
5630,liber,39559,65900,38496,53100,49263.75,True,True,True,"book, volume, inner bark of a tree, book, book"
4259,res,29069,70669,28291,51565,44898.50,True,True,True,suddenly
1219,uideo,21536,59099,26372,50503,39377.50,True,True,True,
5787,locus,23102,56782,31421,43680,38746.25,True,True,True,"place, location"
1257,corpus,16979,63171,22871,31938,33739.75,True,True,True,"body, corpus"


In [4]:
vocab_freqs = filtered_vocab_df.set_index("word")["1601-1650"].to_dict()

# FastText - development and testing

In [10]:
# a genrator for iteration without flooding the CPU
class SentsCorpus:
    def __iter__(self):
        source_path = "/srv/data/tome/noscemus/sents_data/"
        for id in ids:
            f_sents_data = pickle.load(open(source_path + str(id) + ".pickle", "rb"))
            sents_n = len(f_sents_data)
            tokens_n = 0
            lemmata = []
            for sent_data in f_sents_data:
                tokens_n += len(sent_data[1])
                sent_lemmata = [t[1] for t in sent_data[1] if t[2] in ["NOUN", "VERB", "ADJ", "PROPN"]]
                sent_lemmata = [re.sub("\W*|\d*", "", t) for t in sent_lemmata]
                sent_lemmata = [l.lower() for l in sent_lemmata if len(l) > 1]
                try:
                    yield sent_lemmata
                except:
                    pass

In [7]:
# a genrator for iteration without flooding the CPU
class SentsCorpusBackup:
    def __iter__(self):
        for id in ids:
            with open("../data/large_data/sents_lemmata/{}.txt".format(str(id)), "r") as f:
                sents_lemmata = [sent.strip().split() for sent in f.readlines()]
            with open("../data/large_data/sents_pos/{}.txt".format(str(id)), "r") as f:
                sents_pos = [sent.strip().split() for sent in f.readlines()]
            sents_lemmata_filtered = [] # initiate a new empty list for sentence lemmata filtered by POS tags 
            for n in range(len(sents_lemmata)):
                try:
                    pos_indices = [t[0] for t in enumerate(sents_pos[n]) if t[1] in ["PROPN", "NOUN","VERB", "ADJ"]]
                    sent_filtered = [sents_lemmata[n][i] for i in pos_indices]
                    sent_filtered = [re.sub("\W*|\d*", "", t) for t in sent_filtered]
                    yield [t for t in sent_filtered if len(t)>1]
                except:
                    pass

In [12]:
# test with a small corpus from one decade
ids = metadata_table_long[metadata_table_long["file_year"].between(1601, 1610)]["id"]
vocab_freqs = filtered_vocab_df.set_index("word")["1601-1650"].to_dict()
corpus = SentsCorpus()

In [13]:
# how many sentences are there in the small corpus?
len([s for s in corpus])

194843

In [14]:
%%time
# let's train the model
model = FastText(vector_size=100, window=10, negative=25, ns_exponent=1, sg=1, epochs=15, workers=8, min_n=5)
model.build_vocab_from_freq(word_freq=vocab_freqs)
model.train(corpus, total_examples=len([s for s in corpus]), epochs=model.epochs)

CPU times: user 4min 3s, sys: 2.24 s, total: 4min 5s
Wall time: 1min 8s


(10942294, 16475280)

In [18]:
# most similar words to harmonia
model.wv.most_similar("harmonia")

[('concentus', 0.6198955178260803),
 ('inuiolabilis', 0.5702183246612549),
 ('conuenientus', 0.5379429459571838),
 ('compositus', 0.5328949689865112),
 ('compositio', 0.5327402353286743),
 ('accentus', 0.5318533182144165),
 ('concessio', 0.5280140042304993),
 ('incompositus', 0.5252606868743896),
 ('compono', 0.5224994421005249),
 ('structura', 0.5185607075691223)]

# FastText - training the main models

In [20]:
os.mkdir("/srv/data/tome/noscemus/vectors/")

In [21]:
periods = [(1501, 1550),
                      (1551, 1600),
                      (1601, 1650),
                      (1651, 1700)]
periods_str = filtered_vocab_df.columns[1:5]
periods_str

Index(['1501-1550', '1551-1600', '1601-1650', '1651-1700'], dtype='object')

In [22]:
%%time
# main training of the models
for per_tup, per_str in zip(periods, periods_str):
    ids = metadata_table_long[metadata_table_long["file_year"].between(per_tup[0], per_tup[1])]["id"]
    vocab_freqs = filtered_vocab_df.set_index("word")[per_str].to_dict()
    corpus = SentsCorpus()
    model = FastText(vector_size=100, window=10, negative=25, ns_exponent=1, sg=1, epochs=15, workers=32, min_n=5)
    model.build_vocab_from_freq(word_freq=vocab_freqs)
    model.train(corpus, total_examples=len([s for s in corpus]), epochs=model.epochs)
    print("model for the period {} successfully trained.".format(per_str))
    model.wv.save("/srv/data/tome/noscemus/vectors/vectors_{}.wv".format(per_str))

model for the period 1501-1550 successfully trained.
model for the period 1551-1600 successfully trained.
model for the period 1601-1650 successfully trained.
model for the period 1651-1700 successfully trained.
CPU times: user 2h 18min 39s, sys: 57.3 s, total: 2h 19min 37s
Wall time: 40min 48s


In [23]:
# if you want to upload the vectors to sciencedata:
#shutil.make_archive("../data/large_data/vectors", 'zip', "../data/large_data/vectors") #
#s.s.put("https://sciencedata.dk/sharingout/kase%40zcu.cz/TOME/DATA/NOSCEMUS/vectors.zip", data=open("../data/large_data/vectors.zip", "rb"))
# shutil.make_archive("../data/large_data/lila_vectors", 'zip', "../data/large_data/lila_vectors") #
# s.s.put("https://sciencedata.dk/sharingout/kase%40zcu.cz/TOME/DATA/NOSCEMUS/lila_vectors.zip", data=open("../data/large_data/lila_vectors.zip", "rb"))

In [26]:
periods_str = filtered_vocab_df.columns[1:5]
vectors_dict = {
    "lasla" : KeyedVectors.load_word2vec_format("/srv/data/lila/allLASLA-lemmi-fast-100-SKIP-win5-min5.vec"),
    "operamaiora" : KeyedVectors.load_word2vec_format("/srv/data/lila/opera-maiora-lemmas_skip_100.vec")
}
for per_str in periods_str:
    vectors_dict[per_str] = KeyedVectors.load("/srv/data/tome/noscemus/vectors/vectors_{}.wv".format(per_str))

In [30]:
with open("/srv/data/tome/noscemus/vectors/vectors_dict.pkl", "wb") as file:
    pickle.dump(vectors_dict, file)

In [28]:
#s.s.put("https://sciencedata.dk/sharingout/kase%40zcu.cz/TOME/DATA/NOSCEMUS/vectors_dict.pkl", data=open("../data/large_data/vectors_dict.pkl", "rb"))

# Compressing the vectors in vector_dict

In [42]:
#with open("/srv/data/tome/noscemus/vectors/vectors_dict.pkl", "rb") as file:
#    vectors_dict = pickle.load(file)

In [36]:
key = "1601-1650"
model = vectors_dict[key]

word_vectors = model.vectors
vocabulary = model.index_to_key
kv = KeyedVectors(vector_size=word_vectors.shape[1])

# Fill the keyed vectors
for word, vector in zip(vocabulary, word_vectors):
    kv.add_vector(word, vector)

In [37]:
model.most_similar("anima")

[('animo', 0.7404363751411438),
 ('uiuo', 0.7070033550262451),
 ('semen', 0.6555984616279602),
 ('forma', 0.6430976986885071),
 ('immortalis', 0.6415685415267944),
 ('materia', 0.6383398771286011),
 ('animal', 0.6124547123908997),
 ('generatio', 0.6117553114891052),
 ('uitalis', 0.6035860776901245),
 ('animalis', 0.5980802178382874)]

In [39]:
vectors_dict_comp = {}
for key in vectors_dict.keys():
    word_vectors = vectors_dict[key].vectors
    vocabulary = vectors_dict[key].index_to_key
    kv = KeyedVectors(vector_size=word_vectors.shape[1])
    # Fill the keyed vectors
    for word, vector in zip(vocabulary, word_vectors):
        kv.add_vector(word, vector)
    vectors_dict_comp[key] = kv

In [40]:
vectors_dict_comp.keys()

dict_keys(['lasla', 'operamaiora', '1501-1550', '1551-1600', '1601-1650', '1651-1700'])

In [41]:
vectors_dict_comp["1501-1550"].most_similar("scientia")

[('cognitio', 0.7737149000167847),
 ('mathematica', 0.755704402923584),
 ('philosophia', 0.6898273825645447),
 ('disciplina', 0.6702460646629333),
 ('ars', 0.647391676902771),
 ('doctrina', 0.6472513675689697),
 ('sapientia', 0.6469572186470032),
 ('necessarius', 0.6430428624153137),
 ('mathematicus', 0.6296356916427612),
 ('geometria', 0.6173121333122253)]

In [43]:
with open("/srv/data/tome/noscemus/vectors/vectors_dict_comp.pkl", "wb") as file:
    pickle.dump(vectors_dict_comp, file)